In [3]:
#autoreload
%load_ext autoreload
%autoreload 2

import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

import sys
sys.path.append("../../src_clasificacion_vistas/common")

import pycimg

from pathlib import Path
import cv2
from tqdm import tqdm

import shutil

In [4]:




# buscar recursivamente ficheros lista_a.txt
def buscar_ficheros_lista_a(directorio):
    ficheros = []
    for raiz, dirs, archivos in os.walk(directorio):
        for archivo in archivos:
            if archivo == "lista_a.txt" or archivo == "lista_b_a.txt":
                ficheros.append(os.path.join(raiz, archivo))
    return ficheros

def buscar_ficheros_json(directorio):
    ficheros = []
    for raiz, dirs, archivos in os.walk(directorio):
        for archivo in archivos:
            if ".json" in archivo:
                ficheros.append(os.path.join(raiz, archivo))

    return ficheros

In [ ]:
dir_repaso = "/home/aalbiol/Desktop/out"
ficheros_lista_a = buscar_ficheros_json(dir_repaso)
ficheros_lista_a = sorted(ficheros_lista_a)
print(ficheros_lista_a)
print(len(ficheros_lista_a))

[]
0


In [27]:
def clase_y_etiqueta(fich_lista_a):
    partes=fich_lista_a.split("/")
    clase=partes[-2]
    label=partes[-3]
    if label=='positive':
        label=1
    elif 'negative' in label:
        label=0

    return clase, label

def json_y_vista(fichjpg):
    basename=os.path.basename(fichjpg)
    basename=basename.replace(".jpg", "")
    partes=basename.split("_")
    score=partes[-1]
    vista=partes[-2]
    jsonfile="_".join(partes[:-2])+".json"

    return jsonfile, vista, score

In [28]:
jsons_originales=buscar_ficheros_json("../data")
print(len(jsons_originales))

lista_originales=[]
for f in jsons_originales:
    print(f)
    with open(f, 'r') as fjson:
        data = json.load(fjson)
        lista_originales.append((f,data))

1626
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000013.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1651_002016.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000015.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1651_002012.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000019.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1651_000009.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1651_002013.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000014.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000017.json
../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_002018.

In [29]:
lista_originales[:3]

[('../data/mandarins/clementine/KDC/20220803_230825_CLEMENTINE_Encoder_Geens/20250217_1652_000013.json',
  {'information': {'BatchName': 'Mandarines',
    'BatchType': 'Clementine',
    'BatchCoop': 'KDC',
    'Sequence': '20220803 230825 CLEMENTINE_Encoder_Geens'},
   'maxValChannels': 1023,
   'channels': 'H;S;I;NIR;UV;R;G;B;AuxB1',
   'annotations': {'bleaching': '0',
    'cracks': '0',
    'damage_by_pliers': '0',
    'deformed_counterpeduncle': '0',
    'deformed_peduncle': '0',
    'green': '1',
    'hollow': '0',
    'insect': '0',
    'light_scars': '0',
    'mechanical_damage': '0',
    'oil_spots': '0',
    'plu_stickers': '0',
    'reddish': '0',
    'rotten': '0',
    'scars': '0',
    'skin_breakdown': '0',
    'skin_creasing': '0',
    'surface': '0'},
   'files': {'1': '20250217_1652_000013.png', '2': '20250217_1652_000013.npz'},
   'views_annotations': {'view_0': {'bleaching': '0',
     'cracks': '0',
     'damage_by_pliers': '0',
     'deformed_counterpeduncle': '0',
 

In [30]:
for fichero_lista_a in ficheros_lista_a:
    clase,label=clase_y_etiqueta(fichero_lista_a)
    print(fichero_lista_a,clase,label)
    with open(fichero_lista_a, "r") as f:
        lineas = f.readlines()
    lineas = [x.strip() for x in lineas]
    for linea in lineas:
        jsonfile, vista, score = json_y_vista(linea)
        print(linea, jsonfile, vista, score)
    break

../data/repasoD/negative/cracks/lista_b_a.txt cracks 0
20250217_1713_000053_05_79.jpg 20250217_1713_000053.json 05 79
20250217_1713_000053_06_89.jpg 20250217_1713_000053.json 06 89
20250218_1100_000435_10_75.jpg 20250218_1100_000435.json 10 75
20250224_0906_002153_00_101.jpg 20250224_0906_002153.json 00 101


In [31]:
def procesa_incidencia(out_dir,jsonfile,vista,clase,original_label):
    jsonfile=os.path.join(out_dir,jsonfile)
    if not os.path.exists(jsonfile):
        print(">>>>>>>>>>>> NO existe ", jsonfile)
        return
    with open(jsonfile, 'r') as fjson:
        data = json.load(fjson)

        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        print(clase,vista,original_label)
        print(jsonfile)
        views_anots=data["views_annotations"]
        fruit_anots=data["annotations"]
        print("Fruit anots", fruit_anots)
        print("Antes vista ",views_anots['view_'+str(int(vista))])
        view_anots=views_anots['view_'+str(int(vista))]
        original_label=int(original_label)
        new_label=1-original_label
        new_label=str(new_label)
        view_anots[clase]=new_label
        print("Despues vista ",views_anots['view_'+str(int(vista))])
        class_views_labels=[]
        for k,v in views_anots.items():
            class_views_labels.append(int(v[clase]))
        class_views_labels=np.array(class_views_labels)
        class_fruit_label=str(np.max(class_views_labels))
        fruit_anots[clase]=class_fruit_label
        print("New Fruit anots", fruit_anots)

        with open(jsonfile, 'w') as fjson:
            json.dump(data, fjson, indent=4)

def generar_copias_jsons(lista_originales,out_dir,jsonfile):
    for f, data in lista_originales:
        if jsonfile in f:            
            target_jsonfile=os.path.join(out_dir,os.path.basename(f))
            print("Copiando ", f, " a ", target_jsonfile)
            shutil.copyfile(f, target_jsonfile)


In [32]:
### Recargar originales
jsons_originales=buscar_ficheros_json("../data")
print(len(jsons_originales))

lista_originales=[]
for f in jsons_originales:
    #print(f)
    with open(f, 'r') as fjson:
        data = json.load(fjson)
        lista_originales.append((f,data))

### Determinar JSONS a cambiar
nincidencias=0

set_ficheros_a_modificar=set()
for fichero_lista_a in ficheros_lista_a:
    clase,label=clase_y_etiqueta(fichero_lista_a)
    print(fichero_lista_a,clase,label)
    with open(fichero_lista_a, "r") as f:
        lineas = f.readlines()
    lineas = [x.strip() for x in lineas]
    for linea in lineas:
        jsonfile, vista, score = json_y_vista(linea)
        print(linea, jsonfile, vista, score)
        set_ficheros_a_modificar.add(jsonfile)
        nincidencias+=1

print("N incidencias", nincidencias)
print("N ficheros a modificar", len(set_ficheros_a_modificar))



1626
../data/repasoD/negative/cracks/lista_b_a.txt cracks 0
20250217_1713_000053_05_79.jpg 20250217_1713_000053.json 05 79
20250217_1713_000053_06_89.jpg 20250217_1713_000053.json 06 89
20250218_1100_000435_10_75.jpg 20250218_1100_000435.json 10 75
20250224_0906_002153_00_101.jpg 20250224_0906_002153.json 00 101
../data/repasoD/negative/damage_by_pliers/lista_b_a.txt damage_by_pliers 0
20250218_1149_000558_06_261.jpg 20250218_1149_000558.json 06 261
20250218_1145_000531_08_744.jpg 20250218_1145_000531.json 08 744
20250218_1145_000531_07_483.jpg 20250218_1145_000531.json 07 483
20250218_1120_002342_10_344.jpg 20250218_1120_002342.json 10 344
20250220_1200_000053_10_351.jpg 20250220_1200_000053.json 10 351
20250220_1200_000053_09_442.jpg 20250220_1200_000053.json 09 442
20250220_1239_002144_09_275.jpg 20250220_1239_002144.json 09 275
20250220_1239_002144_10_416.jpg 20250220_1239_002144.json 10 416
20250220_1245_000152_04_707.jpg 20250220_1245_000152.json 04 707
20250220_1317_000176_03_23

In [33]:
# Copiando los originales que se modificarán a otra carpeta
for jsonfile in set_ficheros_a_modificar:
    generar_copias_jsons(lista_originales,out_dir=out_dir,jsonfile=jsonfile)

nincidencias=0

# Modificar las copias
for fichero_lista_a in ficheros_lista_a:
    clase,label=clase_y_etiqueta(fichero_lista_a)
    print(fichero_lista_a,clase,label)
    with open(fichero_lista_a, "r") as f:
        lineas = f.readlines()
    lineas = [x.strip() for x in lineas]
    for linea in lineas:
        jsonfile, vista, score = json_y_vista(linea)
        print(linea, jsonfile, vista, score)

        procesa_incidencia(out_dir,jsonfile,vista,clase,label)
        nincidencias+=1

print("N incidencias", nincidencias)



Copiando  ../data/mandarins/tango/KDC/20220805_180026_tangoPROD/20250218_1203_000704.json  a  /home/aalbiol/Desktop/out_repaso/20250218_1203_000704.json
Copiando  ../data/mandarins/tango/KDC/20220804_230730_TANGO_encoder_good/20250218_1038_002257.json  a  /home/aalbiol/Desktop/out_repaso/20250218_1038_002257.json
Copiando  ../data/oranges/navel/KDC/20220829_214322_prod_naranja_plu/20250218_1006_000388.json  a  /home/aalbiol/Desktop/out_repaso/20250218_1006_000388.json
Copiando  ../data/mandarins/clementine/KDC/20220808_154209_Img_20220808_154156produccionClementina/20250218_1144_000520.json  a  /home/aalbiol/Desktop/out_repaso/20250218_1144_000520.json
Copiando  ../data/mandarins/clementine/KDC/20220803_231126_CLEMENTINE_Encoder_Scar/20250217_1712_002040.json  a  /home/aalbiol/Desktop/out_repaso/20250217_1712_002040.json
Copiando  ../data/mandarins/tango/KDC/20220805_180026_tangoPROD/20250218_1201_002456.json  a  /home/aalbiol/Desktop/out_repaso/20250218_1201_002456.json
Copiando  ../d